# Setting the fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Del = (comms[1]*comms[2]*comms[3]*Word.new('s')).freeze
p 'Del = ' + Del.show

DelPerms = []
perm = Del.dup
(Del.size).times do |n|
  perm = perm.cyclic_permutation
  DelPerms << perm << perm.inverse
end
DelPerms.freeze
p DelPerms.map(&:show)
nil

"Del = (((abAB)(cdCD))(efEF))(s)"
["bABcdCDefEFsa", "ASfeFEdcDCbaB", "ABcdCDefEFsab", "BASfeFEdcDCba", "BcdCDefEFsabA", "aBASfeFEdcDCb", "cdCDefEFsabAB", "baBASfeFEdcDC", "dCDefEFsabABc", "CbaBASfeFEdcD", "CDefEFsabABcd", "DCbaBASfeFEdc", "DefEFsabABcdC", "cDCbaBASfeFEd", "efEFsabABcdCD", "dcDCbaBASfeFE", "fEFsabABcdCDe", "EdcDCbaBASfeF", "EFsabABcdCDef", "FEdcDCbaBASfe", "FsabABcdCDefE", "eFEdcDCbaBASf", "sabABcdCDefEF", "feFEdcDCbaBAS", "abABcdCDefEFs", "SfeFEdcDCbaBA"]


In [5]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


# The Algorithm

In [6]:
w = Word.new('abDd')
w = w*DelPerms[5].split(9)[0]
p w.show
nil

"(abDd)(aBASfeFEd)"


### (i) Contracting the word

In [7]:
# (i) contract
reduced = w.contract.dup
reduced.show

"abaBASfeFEd"

### (ii) removing subwords

In [8]:
def reduce_once(myw)
  facs = myw.flatten.factors
  (myw.size+1).times do |i|
    w1 = Word.new(facs[0,i])
    w2 = Word.new(facs.drop(i))
    original_size = myw.size
    for perm in DelPerms
      wc = (w1*perm*w2).deep_copy.contract
      if (wc.size < original_size) then
        myw = wc 
        break
      end
    end
    if (myw.size < original_size) then
      return myw
      break
    end
  end
  return nil    
end
    
def reduce(myword)
  while not myword.nil?
    prev = myword
    myword = reduce_once(myword)
  end
  return prev.contract
end
nil

In [9]:
p 'w = ' + w.show + ' and w.size = ' + w.size.to_s
p reduce(w).show

"w = abaBASfeFEd and w.size = 11"
"acdC"


"acdC"

### (iii) removing subwords part2

In [10]:
# not necessary?

## The algorithm

In [11]:
gens_ex = Gens + [Letter.new('s')]
for g in gens_ex[1..-1]
  p g.show + ': ' + w.count(g).to_s
  p g.inverse.show + ': ' + w.count(g.inverse).to_s
end;nil

"a: 2"
"A: 1"
"b: 1"
"B: 1"
"c: 0"
"C: 0"
"d: 1"
"D: 0"
"e: 1"
"E: 1"
"f: 1"
"F: 1"
"s: 0"
"S: 1"


In [12]:
def N(w, gens)
  gens.map{|g| w.count(g)}.inject(0, &:+)
end;nil

In [13]:
def P(w)
  count = 0
  facs = w.flatten.factors
  facs.each_with_index do |f, i|
    succ = (i < facs.size - 1) ? facs[i+1] : facs[0]
    pair = [f.inverse, succ]
    # p pair.map(&:show)
    idxs = pair.map{|f| Rseq.find_index(f)}
    # p idxs
    count += 1 if idxs[0] < idxs[1]
  end
  return count
end

:P

In [14]:
myw = Word.new(a[1], b[2].inverse).contract
subs = myw.split(1)
ww = subs[0]*(subs[1].inverse)
p "w = #{myw.show}, div = #{subs.map(&:show)}, ww = #{ww.show}"

ww.flatten!
nil

"w = aD, div = [\"a\", \"D\"], ww = (a)(d)"


In [15]:
def T(w, verbose=false)
  w.flatten!
  ais = Gens[1..-1].each_slice(2).map(&:first).map!(&:inverse)
  bs = Gens[1..-1].each_slice(2).map(&:last)
  si = [Letter.new('S')]
  
  result = P(w)-N(w, ais+bs+si)
  return (!verbose) ? result : result.to_s + "  [P=#{P(w)}, A_bar=#{N(w, ais)}, B=#{N(w, bs)}, S_bar=#{N(w, si)}]"
end

:T

In [16]:
myw = Word.new(a[1], b[2].inverse).contract
subs = myw.split(1)
ww = subs[0]*(subs[1].inverse)
p "w = #{myw.show}, div = #{subs.map(&:show)}, ww = #{ww.show}"

p "T(w) = #{T(ww, true)}"
nil

"w = aD, div = [\"a\", \"D\"], ww = (a)(d)"
"T(w) = 0  [P=1, A_bar=0, B=1, S_bar=0]"


In [17]:
w = Word.new(a[1], b[1], a[1].inverse)
p w.show
p w.first == w.last.inverse
w.factors[1..-2].map(&:show)
w.cyclic_permutation(1).contract.show
w.size

"abA"
true


3

In [18]:
100.times do |e|
  rw = Word.new
  10.times do |c|
    g = (rand(2)==0) ? Gens[rand(1...6)] : Gens[rand(1...6)].inverse
    rw = (rw*Word.new(g)).contract
  end
  rw.factors.delete(Letter.new)
  num = P(rw)+P(rw.inverse)
  if num != rw.size 
    p "word = #{rw.show}, size = #{rw.size}, num = #{num}"
  end
end; nil

"word = abdaBDAA, size = 8, num = 7"
"word = BAcBAEBeCb, size = 10, num = 9"
"word = bdBddB, size = 6, num = 5"
"word = DAEaCAcABd, size = 10, num = 9"
"word = CecbABdEbc, size = 10, num = 9"
"word = bEDCBAADEB, size = 10, num = 9"
"word = cEDC, size = 4, num = 3"
"word = CdEBecAc, size = 8, num = 7"


In [19]:
def examine(myw, verbose=false)
  flag = 0
  results = []
  (myw.size-1).times do |j|
    wp = myw.cyclic_permutation(j).contract
    results[j] = "wp = #{wp.show} \n"

    (1...myw.size).each do |i|
      div = wp.split(i)
      words = [div[0]*(div[1].inverse), div[0].dup, div[1].inverse.dup]
      for subw in words
        wcp = Group::Identity
        while wcp.size < subw.size
          subw.factors = wcp.factors unless wcp == Group::Identity
          wcp = subw.cyclic_permutation.contract
        end
      end
      v0, v1, v2 = T(words[0]), T(words[1]), T(words[2])
      if (verbose or v0 != v1 + v2) then
        flag = 1
        results[j] += "#{div.map(&:show)}  --->  " + \
          "T(#{words[0].show}) = #{v0}, T(#{words[1]}) + T(#{words[2]}) = #{v1} + #{v2} = #{v1+v2}" + \
          "\n"
      end
    end
  end
  results = "SIMPLE" if flag == 0
  return (flag==0) ? "SIMPLE" : results.join('')
end; nil

In [20]:
#myw = Word.new(a[1])*(comms[2].inverse)
myw = Word.new(a[1])*comms[2]
p "w = #{myw.show}"

printf examine(myw)

"w = (a)(cdCD)"
SIMPLE

In [21]:
w1 = Word.new(b[2].inverse, b[1].inverse).contract
w2 = Word.new(a[2], a[1]).contract

myw = w1*w2*(w1.inverse)*(w2.inverse)

p "myw = #{myw.show}"
T(myw, true)

"myw = (((DB)(ca))(bd))(AC)"


"1  [P=5, A_bar=2, B=2, S_bar=0]"

## Experiments

### the words $a_{1}([a_{2},b_{2}]^{c})^{k}$, where $c=b_{1}^{a_{1}}$ and $k \in \mathbb{Z}$

In [22]:
a1 = Word.new(a[1])
k=3
myw = a1*(comms[2]^k); p "myw = #{myw.show}"

printf examine(myw)

"myw = (a)(((cdCD)(cdCD))(cdCD))"
wp = acdCDcdCDcdCD 
["acdCD", "cdCDcdCD"]  --->  T(dcDCa) = -1, T(acdCD) + T(dcDCdcDC) = 2 + -2 = 0
wp = cdCDcdCDcdCDa 
["cdCDcdCD", "cdCDa"]  --->  T(cdCDA) = 1, T(cdCDcdCD) + T(AdcDC) = 2 + -2 = 0
wp = dCDcdCDcdCDac 
["dCDcdCDc", "dCDac"]  --->  T(cdCDA) = 1, T(dCDcdCDc) + T(CAdcD) = 2 + -2 = 0
wp = CDcdCDcdCDacd 
["CDcdCDcd", "CDacd"]  --->  T(cdCDA) = 1, T(CDcdCDcd) + T(DCAdc) = 2 + -2 = 0
wp = DcdCDcdCDacdC 
["DcdCDcdC", "DacdC"]  --->  T(cdCDA) = 1, T(DcdCDcdC) + T(cDCAd) = 2 + -2 = 0
wp = cdCDcdCDacdCD 
["cdCDcdCD", "acdCD"]  --->  T(dCDAc) = 1, T(cdCDcdCD) + T(dcDCA) = 2 + -2 = 0
wp = dCDcdCDacdCDc 
wp = CDcdCDacdCDcd 
wp = DcdCDacdCDcdC 
wp = cdCDacdCDcdCD 
["cdCDa", "cdCDcdCD"]  --->  T(adcDC) = -1, T(cdCDa) + T(dcDCdcDC) = 2 + -2 = 0
wp = dCDacdCDcdCDc 
["dCDac", "dCDcdCDc"]  --->  T(adcDC) = -1, T(dCDac) + T(CdcDCdcD) = 2 + -2 = 0
wp = CDacdCDcdCDcd 
["CDacd", "CDcdCDcd"]  --->  T(adcDC) = -1, T(CDacd) + T(DCdcDCdc) = 2 + -2 = 0


In [23]:
c = Word.new(a[1], b[1], a[1].inverse)

ow = myw.dup.flatten

'''
(k-1).times do |i|
  div = ow.split(5+7*i)
  p div.map(&:show)
  ow = (div[0]*c*div[1]*(c.inverse)).flatten
end
'''
ow = Word.new(a[1])*comms[2]
(k-1).times{|i| ow *= c*comms[2]}
ow *= (c.inverse)^(k-1)

p "ow = #{ow.show} = #{ow.contract.show}"
ow = ow.cyclic_permutation.contract

printf examine(ow)

"ow = ((((a)(cdCD))((abA)(cdCD)))((abA)(cdCD)))((aBA)(aBA)) = acdCDabAcdCDabAcdCDaBBA"
SIMPLE

### the word $a_{1}[a_{2}, a_{3}]$

In [24]:
myw = (Word.new(a[1])*Group.commutator(a[2], a[3])).contract
p "w = #{myw.show}"

printf examine(myw)

"w = aceCE"
wp = aceCE 
["ac", "eCE"]  --->  T(acecE) = 2, T(ac) + T(c) = 1 + 0 = 1
["ace", "CE"]  --->  T(aceec) = 2, T(ace) + T(ec) = 2 + 1 = 3
["aceC", "E"]  --->  T(aceCe) = 2, T(aceC) + T(e) = 1 + 0 = 1
wp = ceCEa 
wp = eCEac 
["eCE", "ac"]  --->  T(eCECA) = -2, T(C) + T(CA) = 0 + -1 = -1
wp = CEace 
["C", "Eace"]  --->  T(CECAe) = -2, T(C) + T(CA) = 0 + -1 = -1
["CE", "ace"]  --->  T(CEECA) = -2, T(CE) + T(ECA) = -1 + -2 = -3


In [25]:
k = 1
div = myw.split(k); p div.map(&:show)

abA = Word.new(a[1], b[1], a[1].inverse)
gamma = Group.commutator(abA, b[1])
ow = div[0]*(gamma*div[1]*(gamma.inverse))
p ow.show
ow.contract
ow = ow.cyclic_permutation(1).contract
p ow.show

'''
#-------------------
div = ow.cyclic_permutation(0).split(1)
p div.map(&:show)

gamma = Group.commutator(a[2], a[3]) #comms[1]
ow = div[0]*(gamma*div[1]*(gamma.inverse))
ow = ow.cyclic_permutation(2).contract
p ow.show
'''
printf "---------------------------\n"
result = examine(ow) 
p "Is the loop simple?  -->  #{result == "SIMPLE"}"; nil

["a", "ceCE"]
"(a)(((abAbaBAB)(ceCE))(babABaBA))"
"abAbaBABceCEbabABaB"
---------------------------
"Is the loop simple?  -->  false"


In [26]:
comm4 = Group.commutator(a[1], a[2])
gamma = Group.commutator(c, a[2])

(comm4.size-1).times do |i|
  div = comm4.split(i+1)
  ow = div[0]*gamma*div[1]*(gamma.inverse)
  p ow.show
  ow = ow.cyclic_permutation(4).contract
  p ow.show
  
  result = examine(ow)
  printf "Is the loop simple?  -->  #{result == "SIMPLE"}\n\n"; nil
end; nil

"(((a)(abAcaBAC))(cAC))(cabACaBA)"
"caBAbACaBabA"
Is the loop simple?  -->  false

"(((ac)(abAcaBAC))(AC))(cabACaBA)"
"AcaBACbACaBcab"
Is the loop simple?  -->  false

"(((acA)(abAcaBAC))(C))(cabACaBA)"
"bAcaBACabACaBc"
Is the loop simple?  -->  false

